In [1]:
import numpy as np
import os
import timeit
import glob 
import pandas as pd
import time
import multiprocessing as mp 
import logging
import sys 
import pathlib
from pathlib import Path
import matplotlib.pyplot as plt 

Matplotlib created a temporary config/cache directory at /scratch/slurm-job.816371/matplotlib-hj6n8fbw because the default path (/cluster/customapps/biomed/grlab/users/prelotla/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Order 
- Uniprot 
- Sample init
- Sample expression 
- Sample cohort 
- annotation 
- GTEX?

In [2]:
def filter_cancer_cohort(df, n_samples, threshold_column ):
    df = df.loc[df[threshold_column] >= n_samples]    
    return df
        

In [3]:
def get_threshold_colname(threshold, tag):
    if (threshold is not None) and (threshold > 0 ):
        threshold_column = f'{tag}filter >={threshold}'
    else:
        threshold_column = f'{tag}filter >{threshold}'
    return threshold_column

In [4]:
def filter_single_col(df, threshold, colname):
    if threshold:
        df = df.loc[df[colname] >= threshold, :]
    else:
        df = df.loc[df[colname] >threshold, :]
    return df

In [5]:
def max_recurrence_over_kmer(df, threshold_column, new_maxcol):
    df = df[['kmer', threshold_column]].groupby('kmer').max()
    df = df.reset_index()
    df = df.rename({threshold_column: new_maxcol}, axis = 1)
    return df 

In [6]:
def output_count(df, report_count, report_step, step_string, perform_count=True):
    '''
    Performs a count operation on the number of kmers present in spark dataframe after a given filtering step
    Note: This operation is expensive but useful if the user is interested in intermediate filtering steps
    :param perform_count: bool whether to perform a count operation
    :param df: dataframe with kmer expression counts
    :param report_count: list to store result of successive counting operations
    :param report_step: list to store name of successive counting operations
    :param step_string: str name of the counting operation
    '''
    if perform_count:
        mycount = df['kmer'].unique().shape[0]
        report_count.append(mycount)
        report_step.append(step_string)
        print(f'# {step_string} n = {mycount} kmers')

In [7]:
def save_output_count(output_count, report_count, report_steps, prefix, cancer_sample_ori, mutation_mode,
                      sample_expr_support_cancer, cohort_expr_support_cancer, n_samples_lim_cancer,
                          cohort_expr_support_normal, n_samples_lim_normal, id_normals):
    '''
    Saves the number of kmers present in spark dataframe after each filtering step in a tabular file
    :param output_count: str path for count file of intermediate filtering steps
    :param report_count: list to store result of successive counting operations
    :param report_step: list to store name of successive counting operations
    :param prefix: str information to be added to the result line in an info column
    :param cancer_sample_ori: str id of target cancer sample which was filtered
    :param mutation_mode: str information about whether mutations where applied or not
    :param sample_expr_support_cancer: float normalized expression threshold for the cancer target sample
    :param cohort_expr_support_cancer: float normalized expression threshold for the cancer cohort
    excluding the target sample
    hich should be met in n samples
    :param n_samples_lim_cancer: int number of cancer samples in which the cancer cohort expression threshold
    should be met
    :param cohort_expr_support_normal: float normalized expression threshold for the normal cohort
    required in any sample (>=1)
    :param n_samples_lim_normal: int number of normal samples in which any number of reads is required (>0)
    :param id_normals: str id of the normal cohort (example gtex)
    '''
    pipeline = 'peptide-based'
    if output_count:
        header = (f'{"sample"}\t{"mutation_mode"}\t{"pipeline"}\t{"min_sample_reads"}\t{"#_of_cohort_samples"}\t'
                  f'{"reads_per_cohort_sample"}\t{"#_normal_samples_allowed"}\t{"normal_cohort_id"}'
                  f'\t{"reads_per_normal_sample"}')
        line =   (f'{cancer_sample_ori}\t{mutation_mode}\t{pipeline}\t{sample_expr_support_cancer}\t{n_samples_lim_cancer}'
                  f'\t{cohort_expr_support_cancer}\t{n_samples_lim_normal}\t{id_normals}'
                  f'\t{cohort_expr_support_normal}')

        for idx in np.arange(len(report_count)):
            header += f'\t{report_steps[idx]}'
            line += f'\t{report_count[idx]}'
        if prefix:
            header += f'\t{"info"}'
            line += f'\t{prefix}'
        header += "\n"
        line += "\n"
        
#         print(header, len(header.split('\t')))
#         print(line, len(line.split('\t')))
        if not os.path.exists(output_count):
            with open(output_count,"w") as f:
                f.write(header)
        with open(output_count, "a") as f:
            f.write(line)
        logging.info(f'Save intermediate info to {output_count}')

In [8]:
# Julianne sample  mutation_mode   pipeline        min_sample_reads        #_of_cohort_samples     reads_per_cohort_sample #_normal_samples_allowed        normal_cohort_id        reads_per_normal_sample motif_filter    Init_cancer     Filter_Sample   Filter_Sample_Cohort    Filter_Sample_Cohort_CohortBackground   Filter_Motif    Filter_Sample_Cohort_CohortBackground_Uniprot


# MAIN

Note: Foreground joined with GTEX table \ 79 genes 
Genes excluded is a list of 215 genes for now. Should be fine 

In [18]:
run_type = 'ov'

In [20]:
# Inputs

if run_type == 'brca':
    target_samples = ['TCGA-C8-A12P-01A-11R-A115-07.all',
                      'TCGA-AO-A0JM-01A-21R-A056-07.all',
                      'TCGA-BH-A18V-01A-11R-A12D-07.all',
                      'TCGA-A2-A0D2-01A-21R-A034-07.all',
                      'TCGA-A2-A0SX-01A-12R-A084-07.all']
    basedir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102'
    intermediate_output = os.path.join(basedir, 'filtering_intermediate/complete_cancer_candidates_.tsv.gz')
elif run_type == 'ov':
    target_samples = ['TCGA-25-1319-01A-01R-1565-13.all',
                      'TCGA-25-1313-01A-01R-1565-13.all',
                      'TCGA-61-2008-01A-02R-1568-13.all',
                      'TCGA-24-1431-01A-01R-1566-13.all',
                      'TCGA-24-2298-01A-01R-1569-13.all']
    basedir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374'
    intermediate_output = os.path.join(basedir, 'filtering_intermediate/complete_cancer_candidates_.tsv.gz')


# Outputs
filtering_id = 'chosen_filters_06March'

output_dir = os.path.join(basedir, 'filtering_samples', filtering_id)
pathlib.Path(output_dir).mkdir(exist_ok=True, parents=True)

In [21]:
# Discussion 02/22 Choices
# BACKGROUND cohorts we do (cohort_reads, sample_number)- KEEP pipeline as such
# cohort_reads=[0,1,3]
# sample_number=[1,2,10]
# FOREGROUND  (cohort_reads, sample_number) means
# cohort_reads=[0,2]
# sample_number(rest of cohort) =[1, 5]

In [22]:
# Parameters
Threshold_target = [0.0]
Threshold_cancer_cohort = [0.0, 2.0, None] # choices = [0.0, 1.0, 2.0, 3.0, 5.0, 10.0]
N_samples_cancer = [1, 5, None] # choices 1 to 1102 for BRCA and 374 for OV   

Threshold_normal_cohort = [0.0, 1.0, 3.0]   # choices = [0.0, 1.0, 2.0, 3.0, 5.0, 10.0]
N_samples_normal = [1, 2, 10] #choices 1 to max number of samples in Normal whitelist

tag_cancer = 'cancerCohort'
tag_normal = 'gtexCohort'


tag_prefix = 'G_'
mutation_mode = 'ref'
save_tag = 'GtexCohort'

metadata_save = ['kmer', 'coord', 'junctionAnnotated', 'readFrameAnnotated']

In [23]:
# Load matrix to be filtered
df_load = pd.read_csv(intermediate_output, sep = '\t')
df_load = df_load.rename({'batch': f'batch_{run_type}'}, axis = 1)
df_load.shape

(45760252, 24)

In [24]:
df_load.head()

,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,cancerCohortfilter >=2.0,cancerCohortfilter >=3.0,cancerCohortfilter >=5.0,cancerCohortfilter >=10.0,TCGA25131901A01R156513all,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated
0,RKSTQMPCT,1218.0,689.0,249.0,89.0,26.0,4.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN
1,RKSTQMPCT,1279.0,274.0,70.0,28.0,5.0,1.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN
2,RKSTQMPCT,7350.0,4622.0,2653.0,1745.0,1126.0,734.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN
3,RKSTQMPCT,2025.0,479.0,96.0,35.0,8.0,1.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN
4,RKSTQMPCT,8641.0,6957.0,4789.0,3211.0,1347.0,191.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN


In [25]:
df_load.shape

(45760252, 24)

In [ ]:
for cancer_sample_ori in target_samples: # TODO update
    # Sample naming
    target_sample = cancer_sample_ori.replace('-', '').replace('.', '')
    cancer_sample_ori = cancer_sample_ori.replace('.all', '')
    print(f'-------- processing {target_sample} -------- \n')
    
    # Summary file for sample
    summary_file = f'{tag_prefix}filtered_df_{cancer_sample_ori}_samp_chrt_norm_mot.tsv'
    summary_path = os.path.join(output_dir, summary_file)
    print(f'Saving to summary file {summary_path}')
    

    df_expr = []
    report_count = [] 
    report_steps = []
    for threshold_target in Threshold_target:
        for threshold_cancer_cohort in Threshold_cancer_cohort:
            for n_samples_cancer in N_samples_cancer:
                for threshold_normal_cohort in Threshold_normal_cohort:
                    for n_samples_normal in N_samples_normal:
                        if (n_samples_cancer is None) ^ (threshold_cancer_cohort is None):
                            continue

                        adjusted_threshold_col = 'tmp_cancer_cohort'
                        max_threshold_col = 'tmp_normal_Nmax_sup{}'.format(threshold_normal_cohort)
                        max_threshold_col_base = 'tmp_normal_Nmax_sup{}'.format(0)


                        df = df_load.copy()
                        # Make correction for number samples passing theshold in cohort. We want to exclude the target sample in counting
                        if (n_samples_cancer is not None) and (threshold_cancer_cohort is not None):
                            df[adjusted_threshold_col] = df[get_threshold_colname(threshold_cancer_cohort, tag_cancer)]
                            df.loc[df[target_sample] >= threshold_cancer_cohort, adjusted_threshold_col] -=1 

                        # Number of kmers expressed in sample 
                        df = filter_single_col(df, 0, target_sample)
                        output_count(df, report_count, report_steps, 'Init_Sample')

                        # Number of kmers >= threshold in sample 
                        df = filter_single_col(df, threshold_target, target_sample)
                        output_count(df, report_count, report_steps, 'Filter_Sample')

                        
                        # Filter for cancer cancer cohort 
                        if (n_samples_cancer is not None) and (threshold_cancer_cohort is not None):
                            df = filter_cancer_cohort(df, n_samples_cancer, adjusted_threshold_col)
                        output_count(df, report_count, report_steps, 'Filter_Sample_Cohort')
                            


                        # Expression in gtex cohort >= threshold 
                        recurrence_custom =  max_recurrence_over_kmer(df, 
                                                                      get_threshold_colname(threshold_normal_cohort, tag_normal), 
                                                                      max_threshold_col)

                        # Expression in gtex cohort > 0  
                        recurrence_custom_base = max_recurrence_over_kmer(df, 
                                                                          get_threshold_colname(0.0, tag_normal),
                                                                          max_threshold_col_base) 




                        # Perform Background filtering 
                        df = df.merge(recurrence_custom, on = 'kmer', how = 'left')
                        df = df.merge(recurrence_custom_base, on = 'kmer', how = 'left')

                        df = df.loc[ ~ ((df[max_threshold_col] >= 1) & (df[max_threshold_col_base] >= n_samples_normal)), :]
                        output_count(df, report_count, report_steps, 'Filter_Sample_Cohort_CohortNormal')

                        #Perform Annotated junctions filtering 
                        df = df[df['isAnnotated'].isna()]
                        output_count(df, report_count, report_steps, 'Filter_Sample_Cohort_CohortNormal_pepAnnot')

#                         # DEV: Exclude genes where GTEX is missing
#                         df = df.loc[df['exclude'].isna()]
#                         output_count(df, report_count, report_steps, 'Filter_Sample_Cohort_CohortNormal_pepAnnot_EXPGTEX')

                        
                        
                        
                        # Save outputs 
                        # outpaths
                        base_path_final = os.path.join(output_dir,
                                                       (f'{tag_prefix}{cancer_sample_ori}_'
                                                        f'SampleLim{threshold_target}'
                                                        f'CohortLim{threshold_cancer_cohort}'
                                                        f'Across{n_samples_cancer}_'
                                                        f'FiltNormals{save_tag}'
                                                        f'Cohortlim{threshold_normal_cohort}'
                                                        f'Across{n_samples_normal}.tsv.gz'))
                        print(f'Saving outputs to: {base_path_final} \n')
                        df.loc[:, metadata_save].to_csv(base_path_final, compression = 'gzip', index = None, sep = '\t')


    save_output_count(summary_path, report_count, report_steps, '', cancer_sample_ori, mutation_mode,
                      threshold_target, threshold_cancer_cohort, n_samples_cancer,
                          threshold_normal_cohort, n_samples_normal, save_tag)

-------- processing TCGA25131901A01R156513all -------- 

Saving to summary file /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_filtered_df_TCGA-25-1319-01A-01R-1565-13_samp_chrt_norm_mot.tsv
# Init_Sample n = 1197827 kmers
# Filter_Sample n = 1197827 kmers
# Filter_Sample_Cohort n = 1194632 kmers
# Filter_Sample_Cohort_CohortNormal n = 132460 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 129003 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-25-1319-01A-01R-1565-13_SampleLim0.0CohortLim0.0Across1_FiltNormalsGtexCohortCohortlim0.0Across1.tsv.gz 

# Init_Sample n = 1197827 kmers
# Filter_Sample n = 1197827 kmers
# Filter_Sample_Cohort n = 1194632 kmers
# Filter_Sample_Cohort_CohortNormal n = 134010 

# Init_Sample n = 1197827 kmers
# Filter_Sample n = 1197827 kmers
# Filter_Sample_Cohort n = 1173359 kmers
# Filter_Sample_Cohort_CohortNormal n = 123127 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 119662 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-25-1319-01A-01R-1565-13_SampleLim0.0CohortLim0.0Across5_FiltNormalsGtexCohortCohortlim3.0Across1.tsv.gz 

# Init_Sample n = 1197827 kmers
# Filter_Sample n = 1197827 kmers
# Filter_Sample_Cohort n = 1173359 kmers
# Filter_Sample_Cohort_CohortNormal n = 123862 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 120397 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-25-1319-01A-01R-1565-13_SampleLim0.0CohortLim0.0Acro

# Init_Sample n = 1197827 kmers
# Filter_Sample n = 1197827 kmers
# Filter_Sample_Cohort n = 1079263 kmers
# Filter_Sample_Cohort_CohortNormal n = 78080 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 74684 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-25-1319-01A-01R-1565-13_SampleLim0.0CohortLim2.0Across5_FiltNormalsGtexCohortCohortlim1.0Across2.tsv.gz 

# Init_Sample n = 1197827 kmers
# Filter_Sample n = 1197827 kmers
# Filter_Sample_Cohort n = 1079263 kmers
# Filter_Sample_Cohort_CohortNormal n = 81966 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 78562 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-25-1319-01A-01R-1565-13_SampleLim0.0CohortLim2.0Across5_

# Init_Sample n = 1052807 kmers
# Filter_Sample n = 1052807 kmers
# Filter_Sample_Cohort n = 1046564 kmers
# Filter_Sample_Cohort_CohortNormal n = 79613 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 76288 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-25-1313-01A-01R-1565-13_SampleLim0.0CohortLim0.0Across1_FiltNormalsGtexCohortCohortlim0.0Across2.tsv.gz 

# Init_Sample n = 1052807 kmers
# Filter_Sample n = 1052807 kmers
# Filter_Sample_Cohort n = 1046564 kmers
# Filter_Sample_Cohort_CohortNormal n = 83699 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 80365 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-25-1313-01A-01R-1565-13_SampleLim0.0CohortLim0.0Across1_

# Init_Sample n = 1052807 kmers
# Filter_Sample n = 1052807 kmers
# Filter_Sample_Cohort n = 1028667 kmers
# Filter_Sample_Cohort_CohortNormal n = 70104 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 66779 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-25-1313-01A-01R-1565-13_SampleLim0.0CohortLim2.0Across1_FiltNormalsGtexCohortCohortlim0.0Across2.tsv.gz 

# Init_Sample n = 1052807 kmers
# Filter_Sample n = 1052807 kmers
# Filter_Sample_Cohort n = 1028667 kmers
# Filter_Sample_Cohort_CohortNormal n = 73838 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 70504 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-25-1313-01A-01R-1565-13_SampleLim0.0CohortLim2.0Across1_

...

# Init_Sample n = 1215713 kmers
# Filter_Sample n = 1215713 kmers
# Filter_Sample_Cohort n = 1215713 kmers
# Filter_Sample_Cohort_CohortNormal n = 174224 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 170802 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-61-2008-01A-02R-1568-13_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across2.tsv.gz 

# Init_Sample n = 1215713 kmers
# Filter_Sample n = 1215713 kmers
# Filter_Sample_Cohort n = 1215713 kmers
# Filter_Sample_Cohort_CohortNormal n = 183296 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 179870 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-61-2008-01A-02R-1568-13_SampleLim0.0CohortLimNon

# Init_Sample n = 1015992 kmers
# Filter_Sample n = 1015992 kmers
# Filter_Sample_Cohort n = 1014511 kmers
# Filter_Sample_Cohort_CohortNormal n = 90193 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 87311 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-24-1431-01A-01R-1566-13_SampleLim0.0CohortLim0.0Across1_FiltNormalsGtexCohortCohortlim3.0Across2.tsv.gz 

# Init_Sample n = 1015992 kmers
# Filter_Sample n = 1015992 kmers
# Filter_Sample_Cohort n = 1014511 kmers
# Filter_Sample_Cohort_CohortNormal n = 93484 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 90602 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-24-1431-01A-01R-1566-13_SampleLim0.0CohortLim0.0Across1_

# Init_Sample n = 1015992 kmers
# Filter_Sample n = 1015992 kmers
# Filter_Sample_Cohort n = 999462 kmers
# Filter_Sample_Cohort_CohortNormal n = 83592 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 80718 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-24-1431-01A-01R-1566-13_SampleLim0.0CohortLim2.0Across1_FiltNormalsGtexCohortCohortlim1.0Across10.tsv.gz 

# Init_Sample n = 1015992 kmers
# Filter_Sample n = 1015992 kmers
# Filter_Sample_Cohort n = 999462 kmers
# Filter_Sample_Cohort_CohortNormal n = 80705 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 77830 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-24-1431-01A-01R-1566-13_SampleLim0.0CohortLim2.0Across1_F

# Init_Sample n = 1015992 kmers
# Filter_Sample n = 1015992 kmers
# Filter_Sample_Cohort n = 1015992 kmers
# Filter_Sample_Cohort_CohortNormal n = 88449 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 85569 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-24-1431-01A-01R-1566-13_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim1.0Across1.tsv.gz 

# Init_Sample n = 1015992 kmers
# Filter_Sample n = 1015992 kmers
# Filter_Sample_Cohort n = 1015992 kmers
# Filter_Sample_Cohort_CohortNormal n = 89303 kmers
# Filter_Sample_Cohort_CohortNormal_pepAnnot n = 86423 kmers
Saving outputs to: /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/chosen_filters_06March/G_TCGA-24-1431-01A-01R-1566-13_SampleLim0.0CohortLimNoneAcr

# Init_Sample n = 1310144 kmers
# Filter_Sample n = 1310144 kmers
# Filter_Sample_Cohort n = 1266018 kmers


In [ ]:
print('hello')

In [ ]:
df_load.shape

In [ ]:
kmer = 'KVDTLGKST'

In [ ]:
df.loc[df['kmer'] == kmer]

In [ ]:
df_load.loc[df_load['kmer'] == kmer,\
            ['kmer', 'coord', 'exclude', 'batch_brca', 'junctionAnnotated', 'readFrameAnnotated', target_sample,  get_threshold_colname(threshold_cancer_cohort, tag_cancer), \
            get_threshold_colname(threshold_normal_cohort, tag_normal), get_threshold_colname(0.0, tag_normal)]]

In [ ]:
df_load.loc[df_load['kmer'] == kmer, 'batch_brca']

In [ ]:
gene_to_batch.loc[(gene_to_batch['batch_brca'] == 54877)]

In [ ]:
foo = df_load.merge(gene_to_batch[['exclude', f'batch_{run_type}']], on = f'batch_{run_type}', how = 'left')

In [ ]:
foo.loc[(foo['kmer'] == kmer)]

In [ ]:
final  =df.loc[ ~ ((df[max_threshold_col] >= 1) & (df[max_threshold_col_base] >= n_samples_normal)), 
       ['kmer', get_threshold_colname(threshold_cancer_cohort, tag_cancer)]]

In [ ]:
final = df

In [ ]:
final.loc[final['cancerCohortfilter >0.0']> 403 , ['kmer', 'tmp_cancer_cohort',f'batch_{run_type}'] ] 

In [ ]:
final['cancerCohortfilter >0.0'].describe()

In [ ]:
final

In [ ]:
df_load.loc[df_load['kmer'] == kmer,\
            [target_sample,  get_threshold_colname(threshold_cancer_cohort, tag_cancer), \
            get_threshold_colname(threshold_normal_cohort, tag_normal), get_threshold_colname(0.0, tag_normal)]]